In [327]:
"""한국은행 데이터
"""
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from tqdm import tqdm
import pymysql

# load .env
load_dotenv()
api_key = os.environ.get("korea_bank_api_key")
ENDPOINT = os.environ.get("dev_db_endpoint")
USER = os.environ.get("dev_db_user")
PASSWORD = os.environ.get("dev_db_password")

# 통계 정보 조회
- https://ecos.bok.or.kr/api/#/DevGuide/DevSpeciflcation
- 주기: 년:A, 반년:S, 분기:Q, 월:M, 반월:SM, 일: D

In [328]:
conn = pymysql.connect(
    host=ENDPOINT,
    user=USER,
    password=PASSWORD,
    database="aresa",
    port=3306
)

cur = conn.cursor(pymysql.cursors.DictCursor)

In [332]:
collect_idx100_df = pd.read_csv("config/한국은행_100대_지표.csv")
# collect_idx100_dfM
SEARCH_START = "1980-01-01"
SEARCH_END = "2022-10-31"
search_start_query = f"select * from date_d where date = '{SEARCH_START}'"
search_end_query = f"select * from date_d where date = '{SEARCH_END}'"

cur.execute(search_start_query)
search_start_dict = dict(cur.fetchall()[0])
cur.execute(search_end_query)
search_end_dict = dict(cur.fetchall()[0])
search_start_list = []
search_end_list = []

for cycle in collect_idx100_df["cycle"]:
    if cycle == "A":
        search_start_list.append(search_start_dict["year"])
        search_end_list.append(search_end_dict["year"])
    elif cycle == "Q":
        search_start_list.append(search_start_dict["quarter"])
        search_end_list.append(search_end_dict["quarter"])
    elif cycle == "M":
        search_start_list.append(search_start_dict["month"])
        search_end_list.append(search_end_dict["month"])
    elif cycle == "D":
        search_start_list.append(search_start_dict["ymd"])
        search_end_list.append(search_end_dict["ymd"])

collect_idx100_df["search_start"] = search_start_list
collect_idx100_df["search_end"] = search_end_list
collect_idx100_df.fillna(value="?", inplace=True)
collect_idx100_df

,통계_순번,분류,통계명,시점,예시값,단위,stat_code,cycle,item_code1,item_code2,item_code3,item_code4,search_start,search_end
0,1,시장금리,한국은행 기준금리,20221029,3.000,%,722Y001,D,0101000,?,?,?,19800101,20221031
1,2,시장금리,콜금리(익일물),20221028,3.047,%,817Y002,D,010101000,?,?,?,19800101,20221031
2,3,시장금리,KORIBOR(3개월),20221031,3.890,%,817Y002,D,010150000,?,?,?,19800101,20221031
3,4,시장금리,CD수익률(91일),20221031,3.960,%,817Y002,D,010502000,?,?,?,19800101,20221031
4,5,시장금리,통안증권수익률(364일),20221031,3.743,%,817Y002,D,010400001,?,?,?,19800101,20221031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,부동산 가격,주택매매가격지수,202209,103.950,2021.6=100,901Y093,M,H69A,R70A,?,?,198001,202210
96,97,부동산 가격,주택전세가격지수,202209,102.328,2021.6=100,901Y094,M,H69A,R70A,?,?,198001,202210
97,98,부동산 가격,지가변동률(전기대비),202208,0.276,%,901Y064,M,P65A,?,?,?,198001,202210
98,99,원자재,"국제유가(Dubai, 현물)",202209,90.100,달러,902Y003,M,4010102,?,?,?,198001,202210


In [333]:
df_all = pd.DataFrame()

for row in tqdm(collect_idx100_df.itertuples()):

    stat_num = row.통계_순번
    stat_nm = row.통계명
    stat_code = row.stat_code
    cycle = row.cycle
    item_code1 = row.item_code1
    item_code2 = row.item_code2
    item_code3 = row.item_code3
    item_code4 = row.item_code4
    search_start = row.search_start
    search_end = row.search_end

    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/100000/{stat_code}/{cycle}/{search_start}/{search_end}/{item_code1}/{item_code2}/{item_code3}/{item_code4}"
    response = requests.get(url)
    data = response.json()

    df = pd.DataFrame(data["StatisticSearch"]["row"])
    df.insert(0, "STAT_100_NUM", stat_num)
    df.insert(1, "STAT_100_NM", stat_nm)
    df["CYCLE"] = cycle
    df_all = pd.concat([df_all, df])

    # 1개 지표만 뽑는지 검증
    s = set()
    for row in df.itertuples():
        v = row.STAT_NAME + str(row.ITEM_CODE1) + str(row.ITEM_CODE2) + str(row.ITEM_CODE3) \
            + str(row.ITEM_CODE4)
        s.add(v)
    if len(s) > 1:
        print(stat_num, ":", s)

100it [00:49,  2.02it/s]


In [334]:
df_all

,STAT_100_NUM,STAT_100_NM,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,ITEM_CODE2,ITEM_NAME2,ITEM_CODE3,ITEM_NAME3,ITEM_CODE4,ITEM_NAME4,UNIT_NAME,TIME,DATA_VALUE,CYCLE
0,1,한국은행 기준금리,722Y001,1.3.1. 한국은행 기준금리 및 여수신금리,0101000,한국은행 기준금리,None,None,None,None,None,None,연%,19990506,4.75,D
1,1,한국은행 기준금리,722Y001,1.3.1. 한국은행 기준금리 및 여수신금리,0101000,한국은행 기준금리,None,None,None,None,None,None,연%,19990507,4.75,D
2,1,한국은행 기준금리,722Y001,1.3.1. 한국은행 기준금리 및 여수신금리,0101000,한국은행 기준금리,None,None,None,None,None,None,연%,19990508,4.75,D
3,1,한국은행 기준금리,722Y001,1.3.1. 한국은행 기준금리 및 여수신금리,0101000,한국은행 기준금리,None,None,None,None,None,None,연%,19990509,4.75,D
4,1,한국은행 기준금리,722Y001,1.3.1. 한국은행 기준금리 및 여수신금리,0101000,한국은행 기준금리,None,None,None,None,None,None,연%,19990510,4.75,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,100,금,902Y003,9.1.6.3. 국제상품가격,4020401,금(현물),None,None,None,None,None,None,$/OZ,202206,1807.27,M
438,100,금,902Y003,9.1.6.3. 국제상품가격,4020401,금(현물),None,None,None,None,None,None,$/OZ,202207,1765.94,M
439,100,금,902Y003,9.1.6.3. 국제상품가격,4020401,금(현물),None,None,None,None,None,None,$/OZ,202208,1711.04,M
440,100,금,902Y003,9.1.6.3. 국제상품가격,4020401,금(현물),None,None,None,None,None,None,$/OZ,202209,1660.62,M


In [335]:
# 검증
s = set()
for row in df_all.itertuples():
    v = row.STAT_NAME + str(row.ITEM_CODE1) + str(row.ITEM_CODE2) + str(row.ITEM_CODE3) \
        + str(row.ITEM_CODE4)
    s.add(v)

print(len(s))

100


In [337]:
df_all.to_csv("data/한국은행/한국은행_100대_통계지표_19800101_20221031.csv", index=False, encoding="utf8")

In [280]:
# s = set()
# for row in collect_idx100_df.itertuples():
#     v = row.stat_code + str(row.item_code1) + str(row.item_code2) + str(row.item_code3) \
#         + str(row.item_code4)
#     s.add(v)